# Introduction
Here we use the data from Tang et al., 2009 to fit the exchange rates and then see how much the predicted internal fluxes fit the measured values.

So in this notebook i will try to do that, for the aerobic, microaerobic and anaerobic condition. 

In [1]:
import pandas as pd

In [2]:
import cameo


In [3]:
import cobra.io
import copy
from cobra.flux_analysis import flux_variability_analysis
from cobra import Metabolite, Reaction

In [4]:
model = cobra.io.read_sbml_model('../model/p-thermo.xml')

## Aerobic
Here I'll fit the aerobic data to the model and see what the fluxes the model predicts are.
Fluxes will be set from the presented measured yields, and normalizing to a glucose uptake rate of 10 mmol/gcdw/h. I will set the bounds based on the average +- the standard deviation indicated. 

Fluxes to set:
- glucose uptake: 10 mmol/gcdw/h
- acetate: 7.6 to 2.5
- lactate:0.3 to 0.1
- ethanol:0 to 0.2
- formate: 0

In [74]:
model_copy = copy.deepcopy(model)

In [75]:
model_copy.reactions.EX_glc__D_e.boounds = (-10,1000)

In [76]:
model_copy.reactions.EX_ac_e.bounds = (2.5,7.6)

In [77]:
model_copy.reactions.EX_lac__L_e.bounds = (0.1,0.3)

In [78]:
model_copy.reactions.EX_etoh_e.bounds = (0,0.2)

In [79]:
model_copy.reactions.EX_for_e.bounds = (0,0)

In [80]:
model_copy.optimize()

,fluxes,reduced_costs
IDPh,1.362081,0.000000e+00
CAT,0.000543,0.000000e+00
PDHam1hi,0.000000,1.802235e-16
HYDA,0.000000,2.069557e-19
MALHYDRO,0.000000,0.000000e+00
...,...,...
C23OSAL,0.000000,0.000000e+00
MHMSAH,0.000000,0.000000e+00
UREA2tabc,0.000000,-3.349796e-03
EX_urea_e,0.000000,0.000000e+00


Now that I've fixed the exchange rates according to what is measured, I will run an FVA at 99% of the growth rate to see what the internal fluxes are.

I would be interested in seeing the internal fluxes only for the ones that are measured in the paper so that we can compare it as part of validation. To do so, I will import the exel sheet with the measured C13 fluxes and add two columns to that with what the model has predicted in the FVA.

In [81]:
df_aer = pd.read_csv('../databases/Tang 2009 fluxes-aerobic.csv')

Then I will make a small script that will, for each reaction in the df, it will record the upper and lower FVA bounds (normalized to 100) and append it to the dataframe. Then I can plot it somehow to compare all the different reactions and see how well the overall model captures the solution space possible here.

In [82]:
#add metabolite
model_copy.add_metabolites(Metabolite(id='glu__L_c_biomass'))

In [83]:
model_copy.metabolites.glu__L_c_biomass.formula = model_copy.metabolites.glu__L_c.formula

In [84]:
#add pulling reaction
model_copy.add_reaction(Reaction(id='glu_biomass'))

In [85]:
model_copy.reactions.glu_biomass.add_metabolites({
    model_copy.metabolites.glu__L_c:-1,
    model_copy.metabolites.glu__L_c_biomass:1
})

In [86]:
#change biomass consumption to the new metabolites
model_copy.reactions.biomass.add_metabolites({
    model_copy.metabolites.glu__L_c:0.322004464285714,
    model_copy.metabolites.glu__L_c_biomass:-0.322004464285714
})

In [87]:
#also need to add the exchange reaction incase the flux doesn't match the defined aa composition in the biomass reaction
#it would error otherwise.
model_copy.add_boundary(model_copy.metabolites.glu__L_c_biomass,type = 'sink', reaction_id = 'glu_sink')

Reaction identifier,glu_sink
Name,sink
Memory address,0x024085cec188
Stoichiometry,glu__L_c_biomass <=> <=>
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [88]:
#set growth to 99% of maximum for FVA
growth = 0.99*model_copy.optimize().objective_value
model_copy.reactions.biomass.bounds = (growth,growth) 

In [89]:
model_copy.optimize()

,fluxes,reduced_costs
IDPh,1.372518,0.0
CAT,0.000547,0.0
PDHam1hi,0.000000,0.0
HYDA,0.000000,-0.0
MALHYDRO,0.000000,0.0
...,...,...
UREA2tabc,0.000000,0.0
EX_urea_e,0.000000,0.0
ASPO6s,0.030025,0.0
glu_biomass,0.000000,0.0


In [90]:
for index, row in df_aer.iterrows():   
    bounds = flux_variability_analysis(model_copy, row['model_id'])
    min_b = bounds['minimum'][0]
    max_b = bounds['maximum'][0]
    df_aer.loc[index,'silico_UB'] = max_b*10
    df_aer.loc[index,'silico_LB'] = min_b*10

In [91]:
#what is the oxygen uptake rate here
flux_variability_analysis(model_copy, model_copy.reactions.EX_o2_e)

,minimum,maximum
EX_o2_e,-32.064174,-31.375769


So our model will use 32-31 mmol/gcdw/h to grow with 10 mmol/gcdw/h glucose. 


Now I can export this data and plot it to do a general comparison if the predicted fluxes fit the measured fluxes.


In [32]:
df_aer.to_csv('../databases/Tang 2009 - aerobic flux prediction_90.csv')

Niko also recommended running a pFBA and seeing if then (wihtout fixing growth rate at anything) we could get a good correlation between measured and predicted fluxes. So I will do that here.

In [92]:
pfba_solution = cobra.flux_analysis.pfba(model_copy)

In [93]:
#set the pfba_factor to 1.01 to find variability of pfba 
#i.e. this corresponds to 99% FVA
pfba_solution_105 = flux_variability_analysis(model_copy,pfba_factor = 1.01)

In [94]:
#make a dataframe & export
for index, row in df_aer.iterrows():  
    pfba = pfba_solution[row['model_id']]
    mini = pfba_solution_105.loc[row['model_id']]['minimum']
    maxi = pfba_solution_105.loc[row['model_id']]['maximum']
    df_aer.loc[index,'pfba'] = pfba*10
    df_aer.loc[index,'pfba_101_min'] = mini*10
    df_aer.loc[index,'pfba_101_max'] = maxi*10

In [95]:
df_aer.to_csv('../databases/Tang 2009 - aerobic pfba.csv')

## Microaerobic
I will do the same as above for the microaerobic data from the paper.

In [96]:
model_copy = copy.deepcopy(model)

In [97]:
model_copy.reactions.EX_glc__D_e.boounds = (-10,1000)

In [98]:
model_copy.reactions.EX_ac_e.bounds = (3.5,4.5)

In [99]:
model_copy.reactions.EX_lac__L_e.bounds = (6.0,7.4)

In [100]:
model_copy.reactions.EX_etoh_e.bounds = (2.4,3.2)

In [101]:
model_copy.reactions.EX_for_e.bounds = (0.8,1.8)

In [102]:
model_copy.optimize()

,fluxes,reduced_costs
IDPh,0.755189,0.000000e+00
CAT,0.000301,-3.081488e-33
PDHam1hi,0.000000,1.987299e-16
HYDA,0.000000,-0.000000e+00
MALHYDRO,0.000000,2.775558e-17
...,...,...
C23OSAL,0.000000,-1.387779e-17
MHMSAH,0.000000,0.000000e+00
UREA2tabc,0.000000,-3.287522e-03
EX_urea_e,0.000000,0.000000e+00


Now that I've fixed the exchange rates according to what is measured, I will run an FVA at 99% of the growth rate to see what the internal fluxes are.

I would be interested in seeing the internal fluxes only for the ones that are measured in the paper so that we can compare it as part of validation. To do so, I will import the exel sheet with the measured C13 fluxes and add two columns to that with what the model has predicted in the FVA.

In [103]:
df_micaer = pd.read_csv('../databases/Tang 2009 fluxes-microaerobic.csv')

Then I will make a small script that will, for each reaction in the df, it will record the upper and lower FVA bounds (normalized to 100) and append it to the dataframe. Then I can plot it somehow to compare all the different reactions and see how well the overall model captures the solution space possible here.

In [104]:
#add metabolite
model_copy.add_metabolites(Metabolite(id='glu__L_c_biomass'))

In [105]:
model_copy.metabolites.glu__L_c_biomass.formula = model_copy.metabolites.glu__L_c.formula

In [106]:
#add pulling reaction
model_copy.add_reaction(Reaction(id='glu_biomass'))

In [107]:
model_copy.reactions.glu_biomass.add_metabolites({
    model_copy.metabolites.glu__L_c:-1,
    model_copy.metabolites.glu__L_c_biomass:1
})

In [108]:
#change biomass consumption to the new metabolites
model_copy.reactions.biomass.add_metabolites({
    model_copy.metabolites.glu__L_c:0.322004464285714,
    model_copy.metabolites.glu__L_c_biomass:-0.322004464285714
})

In [109]:
#also need to add the exchange reaction incase the flux doesn't match the defined aa composition in the biomass reaction
#it would error otherwise.
model_copy.add_boundary(model_copy.metabolites.glu__L_c_biomass,type = 'sink', reaction_id = 'glu_sink')

Reaction identifier,glu_sink
Name,sink
Memory address,0x024085d91ec8
Stoichiometry,glu__L_c_biomass <=> <=>
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [110]:
#set growth to 99% of maximum for FVA
growth = 0.99*model_copy.optimize().objective_value
model_copy.reactions.biomass.bounds = (growth,growth) 

In [111]:
growth

0.3462944183664386

In [112]:
for index, row in df_micaer.iterrows():
    bounds = flux_variability_analysis(model_copy, row['model_id'])
    min_b = bounds['minimum'][0]
    max_b = bounds['maximum'][0]
    df_micaer.loc[index,'silico_UB'] = max_b*10
    df_micaer.loc[index,'silico_LB'] = min_b*10

In [113]:
#what is the oxygen uptake rate here
flux_variability_analysis(model_copy, model_copy.reactions.EX_o2_e)

,minimum,maximum
EX_o2_e,-14.847924,-14.474642


So now the oxygen consumption under these conditions is predicted to be half that of the aerobic condition.

In [50]:
df_micaer.to_csv('../databases/Tang 2009 - microaerobic flux prediction_90.csv')

Also do pFBA for these conditions

In [114]:
pfba_solution = cobra.flux_analysis.pfba(model_copy)

In [115]:
#set the pfba_factor to 1.01 to find variability of pfba 
#i.e. this corresponds to 99% FVA
pfba_solution_105 = flux_variability_analysis(model_copy,pfba_factor = 1.01)

In [116]:
#make a dataframe & export
for index, row in df_micaer.iterrows():  
    pfba = pfba_solution[row['model_id']]
    mini = pfba_solution_105.loc[row['model_id']]['minimum']
    maxi = pfba_solution_105.loc[row['model_id']]['maximum']
    df_micaer.loc[index,'pfba'] = pfba*10
    df_micaer.loc[index,'pfba_101_min'] = mini*10
    df_micaer.loc[index,'pfba_101_max'] = maxi*10

In [117]:
df_micaer.to_csv('../databases/Tang 2009 - microaerobic pfba.csv')

## Anaerobic
Finally, I will do the same for the anaerobic data.

In [118]:
model_copy = copy.deepcopy(model)

In [119]:
model_copy.reactions.EX_glc__D_e.bounds = (-10,1000)

In [120]:
model_copy.reactions.EX_ac_e.bounds = (5.1,7.1)

In [121]:
model_copy.reactions.EX_lac__L_e.bounds = (8.3,9.5)

In [122]:
model_copy.reactions.EX_etoh_e.bounds = (3.1,4.5)

In [123]:
model_copy.reactions.EX_for_e.bounds = (8.9,11.7)

In [124]:
model_copy.optimize()

,fluxes,reduced_costs
IDPh,0.399989,-2.168404e-19
CAT,0.000159,0.000000e+00
PDHam1hi,0.000000,-4.383961e-16
HYDA,0.000000,9.209529e-17
MALHYDRO,0.000000,0.000000e+00
...,...,...
C23OSAL,0.000000,-1.387779e-17
MHMSAH,0.000000,0.000000e+00
UREA2tabc,0.000000,-3.287525e-03
EX_urea_e,0.000000,0.000000e+00


Now that I've fixed the exchange rates according to what is measured, I will run an FVA at 99% of the growth rate to see what the internal fluxes are.

I would be interested in seeing the internal fluxes only for the ones that are measured in the paper so that we can compare it as part of validation. To do so, I will import the exel sheet with the measured C13 fluxes and add two columns to that with what the model has predicted in the FVA.

In [125]:
df_anaer = pd.read_csv('../databases/Tang 2009 fluxes-anaerobic.csv')

Then I will make a small script that will, for each reaction in the df, it will record the upper and lower FVA bounds (normalized to 100) and append it to the dataframe. Then I can plot it somehow to compare all the different reactions and see how well the overall model captures the solution space possible here.

In [126]:
#add metabolite
model_copy.add_metabolites(Metabolite(id='glu__L_c_biomass'))

In [127]:
model_copy.metabolites.glu__L_c_biomass.formula = model_copy.metabolites.glu__L_c.formula

In [128]:
#add pulling reaction
model_copy.add_reaction(Reaction(id='glu_biomass'))

In [129]:
model_copy.reactions.glu_biomass.add_metabolites({
    model_copy.metabolites.glu__L_c:-1,
    model_copy.metabolites.glu__L_c_biomass:1
})

In [130]:
#change biomass consumption to the new metabolites
model_copy.reactions.biomass.add_metabolites({
    model_copy.metabolites.glu__L_c:0.322004464285714,
    model_copy.metabolites.glu__L_c_biomass:-0.322004464285714
})

In [131]:
#also need to add the exchange reaction incase the flux doesn't match the defined aa composition in the biomass reaction
#it would error otherwise.
model_copy.add_boundary(model_copy.metabolites.glu__L_c_biomass,type = 'sink', reaction_id = 'glu_sink')

Reaction identifier,glu_sink
Name,sink
Memory address,0x024084e89b88
Stoichiometry,glu__L_c_biomass <=> <=>
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [132]:
#set growth to 99% of maximum for FVA
growth = 0.99*model_copy.optimize().objective_value
model_copy.reactions.biomass.bounds = (growth,growth) 

In [133]:
growth

0.18341672421396374

In [134]:
for index, row in df_anaer.iterrows():
    bounds = flux_variability_analysis(model_copy, row['model_id'])
    min_b = bounds['minimum'][0]
    max_b = bounds['maximum'][0]
    df_anaer.loc[index,'silico_UB'] = max_b*10
    df_anaer.loc[index,'silico_LB'] = min_b*10

In [135]:
#what is the oxygen uptake rate here
flux_variability_analysis(model_copy, model_copy.reactions.EX_o2_e)

,minimum,maximum
EX_o2_e,-4.501737,-4.304027


In [68]:
df_anaer.to_csv('../databases/Tang 2009 - anaerobic flux prediction_90_free.csv')

Also do pFBA for these conditions

In [136]:
pfba_solution = cobra.flux_analysis.pfba(model_copy)

In [137]:
#set the pfba_factor to 1.01 to find variability of pfba 
#i.e. this corresponds to 99% FVA
pfba_solution_105 = flux_variability_analysis(model_copy,pfba_factor = 1.01)

In [138]:
#make a dataframe & export
for index, row in df_anaer.iterrows():  
    pfba = pfba_solution[row['model_id']]
    mini = pfba_solution_105.loc[row['model_id']]['minimum']
    maxi = pfba_solution_105.loc[row['model_id']]['maximum']
    df_anaer.loc[index,'pfba'] = pfba*10
    df_anaer.loc[index,'pfba_101_min'] = mini*10
    df_anaer.loc[index,'pfba_101_max'] = maxi*10

In [139]:
df_anaer.to_csv('../databases/Tang 2009 - anaerobic pfba.csv')

To optimize the model, still quite some oxygen is needed in the anaerobic case. Though it does seem that all the fluxes fit well. I will try to minimize the oxygen that is supplied and see if the model can then still grow.

In [181]:
model_copy2 = copy.deepcopy(model)

In [182]:
model_copy2.reactions.EX_glc__D_e.bounds = (-10,1000)

In [183]:
model_copy2.reactions.EX_ac_e.bounds = (5.1,7.1)

In [184]:
model_copy2.reactions.EX_lac__L_e.bounds = (8.3,9.5)

In [185]:
model_copy2.reactions.EX_etoh_e.bounds = (3.1,4.5)

In [186]:
model_copy2.reactions.EX_for_e.bounds = (8.9,11.7)

Now that I've fixed the exchange rates according to what is measured, I will run an FVA at 99% of the growth rate to see what the internal fluxes are.

I would be interested in seeing the internal fluxes only for the ones that are measured in the paper so that we can compare it as part of validation. To do so, I will import the exel sheet with the measured C13 fluxes and add two columns to that with what the model has predicted in the FVA.

In [187]:
df_anaer = pd.read_csv('../databases/Tang 2009 fluxes-anaerobic.csv')

Then I will make a small script that will, for each reaction in the df, it will record the upper and lower FVA bounds (normalized to 100) and append it to the dataframe. Then I can plot it somehow to compare all the different reactions and see how well the overall model captures the solution space possible here.

In [188]:
#add metabolite
model_copy2.add_metabolites(Metabolite(id='glu__L_c_biomass'))

In [189]:
model_copy2.metabolites.glu__L_c_biomass.formula = model_copy.metabolites.glu__L_c.formula

In [190]:
#add pulling reaction
model_copy2.add_reaction(Reaction(id='glu_biomass'))

In [191]:
model_copy2.reactions.glu_biomass.add_metabolites({
    model_copy2.metabolites.glu__L_c:-1,
    model_copy2.metabolites.glu__L_c_biomass:1
})

In [192]:
#change biomass consumption to the new metabolites
model_copy2.reactions.biomass.add_metabolites({
    model_copy2.metabolites.glu__L_c:0.322004464285714,
    model_copy2.metabolites.glu__L_c_biomass:-0.322004464285714
})

In [193]:
#also need to add the exchange reaction incase the flux doesn't match the defined aa composition in the biomass reaction
#it would error otherwise.
model_copy2.add_boundary(model_copy2.metabolites.glu__L_c_biomass,type = 'sink', reaction_id = 'glu_sink')

Reaction identifier,glu_sink
Name,sink
Memory address,0x019410ffd748
Stoichiometry,glu__L_c_biomass <=> <=>
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [194]:
model_copy2.reactions.EX_o2_e.bounds = (-1,0)

In [195]:
#set growth to 99% of maximum for FVA
growth = 0.99*model_copy2.optimize().objective_value
model_copy.reactions.biomass.bounds = (growth,growth) 

In [196]:
for index, row in df_anaer.iterrows():
    bounds = flux_variability_analysis(model_copy2, row['model_id'])
    min_b = bounds['minimum'][0]
    max_b = bounds['maximum'][0]
    df_anaer.loc[index,'silico_UB'] = max_b*10
    df_anaer.loc[index,'silico_LB'] = min_b*10

In [198]:
growth

0.12673254209650406

In [197]:
df_anaer

,Reaction,an_Flux_X,an_flux_stdev,model_id,UB,LB,silico_UB,silico_LB
0,1.0,-84,2.0,PGI,-80,-88,-9.192756e+01,-91.927563
1,2.0,-15,2.0,G6PDH2r,-11,-19,-7.683470e+00,-7.683470
2,3.0,190,4.0,GAPD,198,182,1.937831e+02,193.783144
3,4.0,-103,14.0,PFL,-75,-131,-1.003906e+02,-100.390628
4,NaN,0,NaN,PDH,0,0,-2.038496e-11,0.000000
5,5.0,61,10.0,PTAr,81,41,5.068769e+01,50.325726
6,6.0,-38,7.0,ACALD,-24,-52,-4.500838e+01,-45.008380
7,7.0,38,7.0,ALCD2x,52,24,4.500000e+01,45.000000
8,8.0,-89,6.0,LDH_L,-77,-101,-8.300000e+01,-83.000000


In [113]:
#what is the oxygen uptake rate here
flux_variability_analysis(model_copy2, model_copy2.reactions.EX_o2_e)

,minimum,maximum
EX_o2_e,-1.0,-1.0


So when we minimize the oxygen that is present (i.e. set it to 0.05mmol/mmol glucose) and we see that the fitting becomes much more stringent that when we don't limit the oxygen uptake. But each flux still falls within the measurement error range of the 13C data, so this is good.

In [149]:
df_anaer.to_csv('../databases/Tang 2009 - anaerobic flux prediction_99_fixed.csv')